<a href="https://colab.research.google.com/github/cheems/Remote-Uploader/blob/main/Remote_Uploader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Remote Uploader__

__Supported filehosts:__
*   MediaFire
*   Zippyshare
*   anonfiles
*   BayFiles



In [ ]:
#@title #__👈 Run this cell to install requirements__##
!echo -n "Installing bayfiles-cli..." && npm i -g bayfiles-cli &> /dev/null && echo " Done!"
!echo -n "Installing anonfile-cli..." && mkdir -p sample_data/anonfile-upload &> /dev/null && git clone https://github.com/awersli99/anonfile-upload.git sample_data/anonfile-upload &> /dev/null && cd sample_data/anonfile-upload &> /dev/null && pip install -r requirements.txt &> /dev/null && echo " Done!"
!echo -n "Installing zippyshare-ul..." && mkdir -p sample_data/zs-ul &> /dev/null && git clone https://github.com/Sorrow446/ZS-UL.git sample_data/zs-ul &> /dev/null && cd sample_data/zs-ul &> /dev/null && pip install -r requirements.txt &> /dev/null && echo " Done!"
!echo -n "Installing zippyshare-dl..." && mkdir -p sample_data/zs-dl &> /dev/null && git clone https://github.com/Sorrow446/ZS-DL.git sample_data/zs-dl &> /dev/null && cd sample_data/zs-dl &> /dev/null && pip install -r requirements.txt &> /dev/null && echo " Done!"
!echo -n "Installing mediafire-ul..." && mkdir -p sample_data/mediafire-ul &> /dev/null && git clone https://github.com/JayBrown/mfcmd.py---MediaFire-uploader.git sample_data/mediafire-ul &> /dev/null && pip install pymediafire &> /dev/null && pip install mediafire &> /dev/null && echo " Done!"

#__Mount cloud storage services with Rclone__


In [ ]:
#@markdown <h3>📝 Note: Run this before using Rclone.</h3>
#@markdown <h4>📝 Upload an rclone profile</4>
Setup_Time_Zone = False 

import os; from google.colab import files; from IPython.display import HTML, clear_output

def upload_conf():
  try:
    display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Please upload the config file of rclone (rclone.conf) from your computer.</h2><br>"))
    UploadConfig = files.upload().keys()
    clear_output(wait=True)
    if len(UploadConfig) == 0:
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
    elif len(UploadConfig) == 1:
      for fn in UploadConfig:
        if os.path.isfile("/content/" + fn) == True:
          os.environ["rclone_conf"] = fn
          !mv -f "$rclone_conf" /root/.rclone.conf
          !chmod 666 /root/.rclone.conf
          if Setup_Time_Zone == True:
            !sudo dpkg-reconfigure tzdata
            clear_output(wait=True)
          if os.path.isfile("/usr/bin/rclone") == True:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">Config has been changed.</h2><br></center>"))
          else:
            !rm -rf /content/sample_data/
            !curl -s https://rclone.org/install.sh | sudo bash
            clear_output(wait=True)
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">Installation has been successfully completed.</h2><br></center>"))
        else:
          return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
    else:
      for fn in UploadConfig:
        os.environ["rclone_conf"] = fn
        !rm -f "$rclone_conf"
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
  except:
    clear_output(wait=True)
    return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))

upload_conf()

In [ ]:
# ============================= FORM ============================= #
#@markdown <h3>📝 Run this to mount the drive with rclone</h3>
rclone_config_name = "mydrive" #@param {type:"string"}
local_mount_location = "/content/udrive/" #@param {type:"string"}
# ============================= FORM ============================= #
import time
import os

# clear nohup
open("nohup.out", 'w').close()


# unmount first

!fusermount -u $local_mount_location 2>/dev/null


# mount without waiting for the command to complete
!mkdir $local_mount_location 2>/dev/null
!nohup rclone mount $rclone_config_name: $local_mount_location --buffer-size 96M & 
  


# Show the output that was written to nohup
time.sleep(3)
f = open(r"nohup.out", "r")
nohupText = f.read()
f.close()


dirs = os.listdir(local_mount_location)

if len(dirs) > 0:
  clear_output(wait=True)
  print("Succeeded. ", str(len(dirs)), "dirs found at", local_mount_location)

else:
  print("\n\nNot succeeded. No files or directories in mounted location. \nCheck your config name and content. If the rclone command was not found, run the cell above.\n\n")
  print("log:\n", nohupText)

# __Download/Upload__

In [ ]:
#@title #__⬇️ Download from filehosts to cloud storage services__#
#@markdown __Supported filehosts:__
#@markdown *   MediaFire (https://download2280.mediafire.com/xxx...)
#@markdown *   Zippyshare (https://www5.zippyshare.com/v/xxx...)
#@markdown *   anonfiles (https://anonfiles.com/xxx...)
#@markdown *   BayFiles (https://bayfiles.com/xxx...)
URL = "" #@param {type:"string"}
Download_Destination = "" #@param {type:"string"}
url, downdir = URL, Download_Destination

def get_file_info(url):
  splited = url.split("/")
  filename=splited[-1]
  splitedfile = filename.split("_")
  splitedfile2="_".join(splitedfile[0:-1])
  modfilename=splitedfile2+"."+splitedfile[-1]
  return filename, modfilename

if "zippyshare" in url:
  !cd sample_data/zs-dl && python3 zs-dl.py -u $url -o $downdir > /content/sample_data/output.txt
  !tail -n 2 sample_data/output.txt
elif "mediafire" in url:
  !wget $url -P $downdir
elif "anonfiles" or "bayfiles" in url:
  filename, modfilename = get_file_info(url)
  !bayfilesdownload $url -o $downdir
  !echo -n "Please correct the file extention on your own: file_txt -> file.txt"
else:
  print("Error: Unsupported filehost")

In [ ]:
#@title #__⬆️ Upload to filehosts from cloud storage services__#
#@markdown __Successfully uploaded max file size:__
#@markdown *   anonfiles (1950MB)
#@markdown *   Zippyshare (500MB)
#@markdown *   BayFiles (99MB)
FileHost = "anonfiles" #@param ["Zippyshare", "anonfiles", "BayFiles"]
File_Path = "" #@param {type:"string"}
filehost, infile = FileHost, File_Path

def get_file_info(filepath):
  splited = filepath.split("/")
  filename=splited[-1]
  pathname="/".join(splited[0:-1])
  return filename, pathname

if filehost == "Zippyshare":
  filename, pathname = get_file_info(infile)
  print("Uploading", filename+"...")
  !cd sample_data/zs-ul && python3 zs-ul.py -f "$infile" #> /content/sample_data/output.txt
  #!tail -n 1 sample_data/output.txt
if filehost == "anonfiles":
  filename, pathname = get_file_info(infile)
  print("Uploading", filename+"...")
  !cd sample_data/anonfile-upload && python3 upload.py "$infile" #> /content/sample_data/output.txt
  #!tail -n 2 sample_data/output.txt
if filehost == "BayFiles":
  filename, pathname = get_file_info(infile)
  print(filename, pathname)
  print("Uploading", filename+"...")
  !cd "$pathname" && bayfilesupload "$filename" #> /content/sample_data/output.txt
  #!tail -n 1 sample_data/output.txt

In [ ]:
#@title #__⬆️ MediaFire Uploader__#
#@markdown The MediaFire API is pretty buggy, and you might receive an error, because the server hasn't finished calculating the file's checksum yet; in that case, wait a couple of seconds (depending on the filesize) and repeat the command: it will not upload, but instead detect the already uploaded file and return the download URL.
Email_Address = "" #@param {type:"string"}
Password = "" #@param {type:"string"}
File_path = "" #@param {type:"string"}
mfemail, mfpassword = Email_Address, Password

def get_file_info(filepath):
  splited = filepath.split("/")
  filename=splited[-1]
  pathname="/".join(splited[0:-1])
  return filename, pathname

filename, pathname = get_file_info(File_path)
print("Uploading", filename+"...")
!cd /content/sample_data/mediafire-ul && python3 mfcmd.py -e $mfemail -p $mfpassword -f $File_path 2>/dev/null
print("Done!")

### 📚 Follow me on GitHub for more: [_@cheems_](https://github.com/cheems)
##### 📦 Repo: [Github](https://github.com/cheems/Remote-Uploader)